In [1]:
import django
import sys
import os
sys.path.append('..')

from django.conf import settings
django.setup()

from campfin_data.models import *
import locale
locale.setlocale(locale.LC_ALL, 'en_US')
from django.db.models import Q, Sum, Count

In [2]:
mtn = ScrapedExpense.objects.filter(filing_committee__name='MASS TRANSIT NOW').order_by('-amount')
for m in mtn[:10]:
    print m.amount, m.description, m.date, m.vendor

800000.000 MEDIA ADVERTISING 2016-08-31 SCREEN STRATEGIES MEDIA
400000.000 MEDIA ADVERTISING 2016-09-23 SCREEN STRATEGIES MEDIA
200000.000 MEDIA ADVERTISING 2016-09-28 SCREEN STRATEGIES MEDIA
120000.000 ONLINE ADVERTISING 2016-10-17 KULLY HALL STRUBLE
100000.000 MEDIA ADVERTISING 2016-10-31 SCREEN STRATEGIES MEDIA
81782.840 DIRECT MAIL PRODUCTION 2016-10-28 NORTHWEST PASSAGE CONSULTING
75000.000 ONLINE ADVERTISING 2016-10-14 KULLY HALL STRUBLE
75000.000 MEDIA ADVERTISING 2016-10-21 SCREEN STRATEGIES MEDIA
75000.000 ONLINE ADVERTISING 2016-10-14 KULLY HALL STRUBLE
74250.000 POLLING 2016-06-17 FAIRBANK MASLIN MAULIN METZ


In [3]:
from campfin_data.utils import *

In [4]:
cands = BallotRaceSummary.objects.filter(race_id=999999)
for cand in cands:
    print cand.ballot_id, cand.ballot_name

999991 Yes
999992 No


In [5]:
grouped_expenses = get_grouped_expenses(mtn)
top_expenses = get_top_expenses(grouped_expenses)

In [6]:
top_expenses

[[u'SCREEN STRATEGIES MEDIA',
  1575000.0,
  5,
  u'MEDIA ADVERTISING',
  u'FAIRFAX',
  u'VA'],
 [u'NORTHWEST PASSAGE CONSULTING',
  393881.58,
  34,
  u'STICKER PRINTING; REMIT PRINTING: OVERNIGHT PRINTING; PRINT ADVERTISING; DIRECT MAIL PRODUCTION; DOORBELLER PRINTING; WEB DESIGN; PRINT DESIGN; GENERAL STRATEGY CONSULTING; LOGO DESIGN; PRINTING; ONLINE ADVERTISING; LOGO DESIGN: JENNERGY; VOTER CONTACT TELEPHONE CALLS; BANNER PRINTING; STICKER PRINTING: OVERNIGHT PRINTING',
  u'SEATTLE',
  u'WA'],
 [u'KULLY HALL STRUBLE',
  338073.0,
  4,
  u'ONLINE ADVERTISING; MEDIA PRODUCTION',
  u'SEATTLE',
  u'WA'],
 [u"PUBLISHER'S MAILING SERVICE", 197799.13, 5, u'POSTAGE', u'TUKWILA', u'WA'],
 [u'FAIRBANK MASLIN MAULIN METZ', 124250.0, 3, u'POLLING', u'OAKLAND', u'CA'],
 [u'NEWMAN PARTNERS',
  85000.0,
  9,
  u'FUNDRAISING CONSULTING',
  u'SEATTLE',
  u'WA'],
 [u'EMC RESEARCH', 48500.0, 1, u'EMC RESEARCHPOLLING', u'OAKLAND', u'CA'],
 [u'PACIFIC PUBLIC AFFAIRS',
  48391.29,
  12,
  u'GENERAL STR

In [7]:
race = BallotRaceSummary.objects.filter(race_id=39920)
candidates = set()
for r in race:
    if r.ballot_id not in candidates:
        print r.race_name, r.district.district_name, r.ballot_name, r.ballot_id
    candidates.add(r.ballot_id)

Park Commissioner Position #4 Proposed Metropolitan Park District Doug Sayan 45736
Park Commissioner Position #4 Proposed Metropolitan Park District Richard Hoss 45738
Park Commissioner Position #4 Proposed Metropolitan Park District Monte Warren Ritter 45746


In [8]:
construction_employers = ['WSP PARSONS BRINCKERHOFF', 'SCHARRER ARCHITECTURE AND DESIGN', 'PARAMETRIX']
construction_firms = ['WASHINGTON ENGINEERS PAC', 'AECOM', 'HNTB CORPORATION', 'CH2M HILL ENGINEERS',
                      'PARSONS TRANSPORTATION GROUP', 'PARAMETRIX INC', 'JACOBS ENGINEERING GROUP INC.', 'HDR INC.',
                      'KPFF CONSULTING ENGINEERS', 'LTK ENGINEERING SERVICES', 'PARSONS BRINCKERHOFF INC',
                      'ENVIROISSUES INC', 'KBA INC', 'GRANITE CONSTRUCTION', 'DAVID EVANS &AMP; ASSOCIATES',
                      'NUCOR STEEL SEATTLE INC.', 'PARSONS BRINCKERHOFF INC.', 'ENVIROISSUES INC.',
                      'HATCH MOTT MACDONALD', 'SHANNON &AMP; WILSON INC', 'PERTEET INC.', 'TOUCHSTONE', 'CDM SMITH',
                      'THE GREENBUSCH GROUP INC.', 'PERKINS &AMP; WILL', 'MITHUN']
construction_labor = ['WASHINGTON AND NORTHERN IDAHO DISTRICT COUNCIL OF LABORERS',
                      'INTERNATIONAL UNION OF OPERATING ENGINEERS',
                      'INTERNATIONAL UNION OF OPERATING ENGINEERS LOCAL 302',
                      'WASHINGTON STATE BUILDING AND CONSTRUCTION TRADES COUNCIL', 'WALECET',
                      'WASHINGTON STATE LABOR COUNCIL AFL-CIO',
                      'HOD CARRIERS &AMP; GENERAL LABORERS UNION LOCAL 242',
                      'CEMENT MASONS &AMP; PLASTERERS LOCAL #528',
                      'IRON WORKERS DISTRICT COUNCIL OF THE PACIFIC NORTHWEST', 'LABORERS LOCAL 440',
                      'NORTHWEST REGIONAL COUNCIL NCA 91',
                      'INTERNATIONAL BROTHERHOOD OF ELECTRICAL WORKERS LOCAL 46',
                      'MLK JR COUNTY LABOR COUNTY COMMITTEE ON POLITICAL EDUCATION']
re_firms = ['VULCAN INC.', 'METZLER REALTY ADVISORS INC. ', 'R.C. HEDREEN CO', 'TALON PRIVATE CAPITAL LLC',
            'AMLI RESIDENTIAL', 'NAIOP COMMERCIAL REAL ESTATE DEVELOPMENT ASSOCIATION', 'UNICO PROPERTIES LLC',
            'KINZER PARTNERS']
misc_labor = ['AEROSPACE MACHINISTS INDUSTRIAL DISTRICT LODGE NO. 751']
tech_firms = ['MICROSOFT', 'AMAZON', 'EXPEDIA', 'MICROSOFT', 'SEATTLEGENETICS', 'CENTURYLINK',]

mtn = ScrapedContribution.objects.filter(filing_committee__name='MASS TRANSIT NOW')
totals = {
    'construction': {
        'sum': 0.,
        'num': 0
    },
    're': {
        'sum': 0.,
        'num': 0
    },
    'tech': {
        'sum': 0.,
        'num': 0
    },
    'other': {
        'sum': 0.,
        'num': 0
    }
}

total = 0.
for m in mtn:
    categorized = False
    total += float(m.amount)
    if m.employer in construction_employers or m.donor in construction_firms or m.donor in construction_labor:
        totals['construction']['sum'] += float(m.amount)
        totals['construction']['num'] += 1
        categorized = True
    """
    if m.donor in construction_labor:
        totals['construction_labor']['sum'] += float(m.amount)
        totals['construction_labor']['num'] += 1
        categorized = True
    if m.donor in misc_labor:
        totals['misc_labor']['sum'] += float(m.amount)
        totals['misc_labor']['num'] += 1
        categorized = True
    """
    if m.donor in re_firms:
        totals['re']['sum'] += float(m.amount)
        totals['re']['num'] += 1
        categorized = True
    if m.donor in tech_firms:
        totals['tech']['sum'] += float(m.amount)
        totals['tech']['num'] += 1
        categorized = True
    if not categorized:
        totals['other']['sum'] += float(m.amount)
        totals['other']['num'] += 1

for key in totals:
    print '%s total: %s (%.2f%% of total) in %d donations' % (
        key.title(),
        locale.currency(totals[key]['sum'], grouping=True),
        (totals[key]['sum']/total) * 100.,
        totals[key]['num'])

Re total: $141,000.00 (3.84% of total) in 7 donations
Construction total: $1,323,250.00 (36.07% of total) in 55 donations
Tech total: $595,000.00 (16.22% of total) in 9 donations
Other total: $1,609,111.00 (43.86% of total) in 274 donations


In [9]:
table_donations = []
for donation in mtn.values(
        'donor', 'employer', 'occupation', 'city', 'state').annotate(
        sum=Sum('amount'), num=Count('donor')).order_by('-sum', '-num', 'donor'):
    industry = []
    if (donation['employer'] in construction_employers or
        donation['donor'] in construction_firms or
        donation['donor'] in construction_labor):
        industry.append('Construction')
    if donation['donor'] in tech_firms:
        industry.append('Technology')
    """
    if donation['donor'] in misc_labor or donation['donor'] in construction_labor:
        industry.append('Labor')
    """
    if donation['donor'] in re_firms:
        industry.append('Real estate')
    table_donations.append({
        'donor': donation['donor'],
        'amount': float(donation['sum']),
        'num': donation['num'],
        'employer': donation['employer'],
        'occupation': donation['occupation'],
        'city': donation['city'].title(),
        'state': donation['state'],
        'industry': '; '.join(industry)
    })

from csv import DictWriter

with open('/home/aepton/code/dataviz/st3_update/contribs.csv', 'w+') as fh:
    writer = DictWriter(fh, table_donations[0].keys())
    writer.writeheader()
    writer.writerows(table_donations)
  
with open('/home/aepton/code/dataviz/st3_update/groups.csv', 'w+') as fh:
    writer = DictWriter(fh, ['industry', 'sum', 'num'])
    writer.writeheader()
    writer.writerows([
        {
            'industry': key,
            'sum': totals[key]['sum'],
            'num': totals[key]['num']
        } for key in totals])

IOError: [Errno 2] No such file or directory: '/home/aepton/code/dataviz/st3_update/contribs.csv'

In [10]:
import datetime

running_totals = {
    'construction': 0.,
    'tech': 0.,
    'real estate': 0.,
    'other': 0.
}
output = []
current = datetime.date(2015, 12, 8)
today = datetime.date.today()
one_day = datetime.timedelta(days=1)

while current <= today:
    donations = mtn.filter(date=current)
    for donation in donations:
        if (donation.employer in construction_employers or
                donation.donor in construction_firms or
                donation.donor in construction_labor):
            running_totals['construction'] += float(donation.amount)
        elif donation.donor in re_firms:
            running_totals['real estate'] += float(donation.amount)
        elif donation.donor in tech_firms:
            running_totals['tech'] += float(donation.amount)
        else:
            running_totals['other'] += float(donation.amount)
    output.append({
        'date': current.strftime('%Y-%m-%d'),
        'construction': running_totals['construction'],
        'tech': running_totals['tech'],
        'real estate': running_totals['real estate'],
        'other': running_totals['other']
    })
    current += one_day

with open('/home/aepton/code/dataviz/st3_update/stream.csv', 'w+') as fh:
    writer = DictWriter(fh, output[0].keys())
    writer.writeheader()
    writer.writerows(output)

IOError: [Errno 2] No such file or directory: '/home/aepton/code/dataviz/st3_update/stream.csv'